In [ ]:
#| default_exp retina_test

In [ ]:
%load_ext autoreload
%autoreload 2

The fundamental question `where is the model failing?`

first thing we need to do is visualize several scans to get an answer to this. 

- Read the cache. 
- Read the scan.
- Visualize the scan alone 
- Visualize Gt bbox 
- Visualize Pred bbox

In [ ]:
#| export
import numpy as np
import cv2
import SimpleITK as sitk 
from voxdet.tfsm.voxt import clip_2_img
from voxdet.utils import hu_to_lung_window

In [ ]:
import torch
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from voxdet.utils import vis, image_grid
from voxdet.metrics.sub_level_analysis import get_all_file_locs
from qct_utils.ctvis.viewer import plot_scans

In [ ]:
scans_root = "/cache/datanas1/qct-nodules/nifti_with_annots/"
folders = ["lidc3_val"]
files = get_all_file_locs(root="../weights/v150/", read_dir=["lidc"])
len(files)

0

In [ ]:
file = files[np.random.randint(len(files))]
scan_loc = Path(scans_root)/ folders[0]/ (file.stem+".nii.gz")
scan_loc.exists()

In [ ]:
img = sitk.ReadImage(scan_loc.as_posix())
array = sitk.GetArrayFromImage(img)
array.shape

In [ ]:
#| export
def load_img(scan_loc, window=True):
    img = sitk.ReadImage(scan_loc.as_posix())
    array = sitk.GetArrayFromImage(img)
    if window: return np.uint8(hu_to_lung_window(array)*255)
    return array

In [ ]:
## Load bboxes
gt_pred = torch.load(file)
gt = gt_pred["img_in"]
pred = gt_pred["img_out"]
gt["boxes"].shape, pred["boxes"].shape, pred["scores"]

In [ ]:
from voxdet.det_metrics import assign_tp_fp_fn_linear_assignment

In [ ]:
tp, fp, fn = assign_tp_fp_fn_linear_assignment(pred["boxes"][pred["scores"]>=0.9], gt["boxes"], 0.1)

In [ ]:
img_lvl = []
for file in tqdm(files):
    gt_pred = torch.load(file)
    gt = gt_pred["img_in"]
    pred = gt_pred["img_out"]
    gt["boxes"].shape, pred["boxes"].shape, pred["scores"]
    tp, fp, fn = assign_tp_fp_fn_linear_assignment(pred["boxes"][pred["scores"]>=0.9], gt["boxes"], 0.1)
    img_lvl.append([file.name, tp.sum(), fp.sum(), fn.sum()])

In [ ]:
df = pd.DataFrame(img_lvl)
df.columns = ["scan_name", "tp", "fp", "fn"]
df.head()

In [ ]:
df[df["fp"] >5]

In [ ]:
df[df["fp"] >5]["scan_name"].values

In [ ]:
lwa = np.uint8(hu_to_lung_window(array)*255)
vis(lwa, 64, window=False)

In [ ]:
gt_boxes = gt["boxes"]
gt_boxes[:, :3] = np.floor(gt_boxes[:, :3])
gt_boxes[:, 3:] = np.ceil(gt_boxes[:, 3:])
gt_boxes = gt_boxes.astype(int)
gt_boxes

In [ ]:
margin = np.asarray([-2, -5, -5, 2, 5, 5])
margin = margin.reshape(1, -1).repeat(gt_boxes.shape[0], axis=0)
margin

In [ ]:
gt_boxes = clip_2_img(gt_boxes+margin, lwa.shape)
gt_boxes

In [ ]:
#| export 
def convert2int(boxes, margin=None, img_shape=None):
    boxes = boxes.copy()
    boxes[:, :3] = np.floor(boxes[:, :3])
    boxes[:, 3:] = np.ceil(boxes[:, 3:])
    boxes = boxes.astype(int)
    if margin is not None:boxes = boxes + np.asarray(margin).reshape(1, -1).repeat(boxes.shape[0], axis=0)
    if img_shape is not None: boxes = clip_2_img(boxes, img_shape)
    return boxes

In [ ]:
#| export 
def draw_bbox(img, bbox, bbox_color: tuple=(255, 0, 0), thickness: int=2, overlay: bool=False, alpha: float=0.5):
    #bbox is is xyxy format
    output = img.copy()
    thickness = -1 if overlay else thickness
    output = cv2.rectangle(output, (bbox[0], bbox[1]), (bbox[2], bbox[3]), bbox_color, thickness)
    if overlay: 
        overlay = img.copy()
        cv2.addWeighted(overlay, alpha, output, 1 - alpha, 0, output)
    return output

In [ ]:
#| export 
def add_label(img, bbox, label, draw_bg=True, text_color=(255, 0, 0), text_bg_color=(255, 255, 255)):
    text_width = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)[0][0]
    tip = +30 if bbox[1]-30<0 else -30 
    label_bg = [bbox[0], bbox[1], bbox[0] + text_width, bbox[1] + tip]
    output = img.copy()
    if draw_bg:cv2.rectangle(output, (label_bg[0], label_bg[1]), (label_bg[2] + 5, label_bg[3]), text_bg_color, -1)
    cv2.putText(output, label, (bbox[0] + 5, bbox[1] - 5 + (30 if tip==30 else 0)), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2)
    return output

In [ ]:
#| export 
def draw_on_ct(img, boxes, color=(255, 0, 0)):
    dimg = img.copy()
    for box in boxes:
        z1, y1, x1, z2, y2, x2 = box
        if z1 == z2: z2 = z2+1
        for z in range(z1, z2):
            img = dimg[z]
            img = draw_bbox(img, (x1, y1, x2, y2), bbox_color=color)
            dimg[z] = img
    return dimg 

In [ ]:
file.parent

In [ ]:
file = files[np.random.randint(len(files))]
#file = file.parent / "1.3.6.1.4.1.55648.166786657465154199470575722567012949663.3.pt"
scan_loc = Path(scans_root)/ folders[0]/ (file.stem+".nii.gz")#rsplit("_")[0]
scan_loc.exists()
lwa = load_img(scan_loc)
dimg = lwa.copy()
dimg = np.concatenate([np.expand_dims(dimg, axis=-1) for _ in range(3)], axis=3)
dimg.shape

In [ ]:
file.stem

In [ ]:
gt_pred = torch.load(file)
gt = gt_pred["img_in"]
pred = gt_pred["img_out"]
#print(gt["boxes"].shape, pred["boxes"].shape, pred["scores"])

## Convert to ints
gt_box = convert2int(gt["boxes"])
pred_box = convert2int(pred["boxes"])
gt_box.shape, pred_box.shape, pred["scores"].shape, (pred["scores"]>0.9).sum()

In [ ]:
from voxdet.bbox_iou import calculate_iou_numpy
iou = calculate_iou_numpy(gt_box, pred_box[pred["scores"]>0.9])
iou,iou.shape

In [ ]:
pred_box[pred["scores"]>0.9]

In [ ]:
gt_box

In [ ]:
print(iou.argmax(1), iou.max(1))

In [ ]:
gtimg = draw_on_ct(dimg, gt_box, (0, 255, 0))
predimg = draw_on_ct(dimg, pred_box[pred["scores"]>0.9], (255, 0, 0))

In [ ]:
plot_scans([gtimg, predimg], ["Gt", "pred"])

In [2]:
#| hide
import nbdev; nbdev.nbdev_export()